## Installing required libraries

- Library for categorical encoding
- weight and biases library

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 3.8 MB/s 


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 180 kB 45.7 MB/s 
     |████████████████████████████████| 139 kB 50.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=9810b8ec756451d47cfb5e42061d592af83931b3f0345f69542db0be33b78eb4
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=9953230b80677a0afb4949ac72d8a836c31713fd8909828114c7eeea3c0fd90b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


## Loading the dataset: Used Car Price Prediction

In [ ]:
import pandas as pd
import numpy as np
from category_encoders import OneHotEncoder, TargetEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import wandb
import os

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
cars_df = pd.read_csv( "https://drive.google.com/uc?export=download&id=1V_VBbyjGj6vvD0A90S5Lk0DG90djz28B" )

In [ ]:
cars_df.head(5)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,mileage_new,engine_new,power_new,age,make,model,KM_Driven
0,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,18.20,1199,88.70,9,honda,jazz,46
1,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,20.77,1248,88.76,8,maruti,ertiga,87
2,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50,23.08,1461,63.10,7,nissan,micra,86
3,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,NaN,1.95,22.30,1248,74.00,8,tata,indica,65
4,Maruti Swift VDI BSIV,Jaipur,2015,64424,Diesel,Manual,First,25.2 kmpl,1248 CC,74 bhp,5.0,NaN,5.60,25.20,1248,74.00,5,maruti,swift,64


In [ ]:
x_columns = ['KM_Driven', 'Fuel_Type', 'age',
              'Transmission', 'Owner_Type', 'Seats',
              'make', 'mileage_new', 'engine_new', 'model',
              'power_new', 'Location']
## model of the car is not included in the model

In [ ]:
cars_df.shape

(3092, 20)

In [ ]:
cars_df = cars_df[x_columns + ['Price']].dropna()

In [ ]:
cars_df.shape

(3091, 13)

## Identifying numerical and categorical features

In [ ]:
cat_features = ['Fuel_Type',
                'Transmission', 'Owner_Type', 'model',
                'make', 'Location']

In [ ]:
num_features = list(set(x_columns) - set(cat_features))

## Utility method for preparing the data

- Splitting the dataset
- Encoding Catgorical Variables

In [ ]:
def prepare_data(X,
                 y,
#                 num_scaler = None,
#                 num_imputer = None,
#                 cat_imputer = None,
                 cat_encoder,
                 num_features,
                 cat_features,
                 train_size = 0.8,
                 seed = 100):
  
  # Split the dataset into train and test split
  x_train, x_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      train_size = train_size,
                                                      random_state = seed)
  
  
  # encoder the categorical features
  cat_encoder.fit(X=x_train, y=y_train)
  x_train_encoded = cat_encoder.transform(x_train)
  x_test_encoded = cat_encoder.transform(x_test)

  return x_train_encoded, x_test_encoded, y_train, y_test
  

In [ ]:
#For removing an environment variable wrongly set by os.environ()
#print(os.environ["WANDB_MODE"])
#os.environ.pop("WANDB_MODE")

## Initilializing Weights and Biases

In [ ]:
os.environ["WANDB_API_KEY"] = "8985894d5778b95c7acb681fd408dffe742c67c3"

In [ ]:
#!wandb online

In [ ]:
def run_model_experiment(model,
                         x_train,
                         y_train,
                         x_test,
                         y_test,
                         name,
                         config = None,
                         tags = None):
  
  model.fit(x_train, y_train)

  wandb.init(project='usedcar', config=config, tags = tags)
  wandb.run.name = name
  rmse = np.sqrt(mean_squared_error(y_test, model.predict(x_test)))
  r2 = model.score(x_test, y_test)

  wandb.log( {"rmse" : rmse, 
              "r2": r2} )
  
  wandb.sklearn.plot_regressor(model, 
                               x_train, 
                               x_test, 
                               y_train, 
                               y_test,  
                               model_name=name)
  
  wandb.Artifact(name, 
                 type = 'model',
                 description = config)

  wandb.save()
  wandb.finish()

  return model

## Baseline Model: Linear Regression

In [ ]:
ohe_encoder = OneHotEncoder(cols = cat_features,
                            use_cat_names = True)

x_train, x_test, y_train, y_test = prepare_data(cars_df[x_columns],
                                                cars_df.Price,
                                                cat_encoder = ohe_encoder,
                                                num_features = num_features,
                                                cat_features = cat_features)

dtree_model = DecisionTreeRegressor(max_depth = 10)  

dtree_model = run_model_experiment(dtree_model,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   name = 'DecisionTreeWithOHE',
                                   tags = ['Decision Tree', 'baseline', 'OheEncoder'])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
wandb: Currently logged in as: manaranjan (use `wandb login --relogin` to force relogin)


wandb: 
wandb: Plotting DecisionTreeWithOHE.
wandb: Logged summary metrics.
wandb: Logged learning curve.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


r2,▁
rmse,▁
r2,0.79775
rmse,0.94576


## Running Multiple Experiments

### Random Forest

In [ ]:
ohe_encoder = OneHotEncoder(cols = cat_features, 
                            use_cat_names = True)

x_train, x_test, y_train, y_test = prepare_data(cars_df[x_columns],
                                                cars_df.Price,
                                                cat_encoder = ohe_encoder,
                                                num_features = num_features,
                                                cat_features = cat_features)

In [ ]:
params = { "n_estimators": 100,
           "max_depth": 10,
           "max_features": .2,
           "max_samples": 0.4 }

In [ ]:
rf_model = RandomForestRegressor(**params)

In [ ]:
rf_model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features=0.2, max_leaf_nodes=None,
                      max_samples=0.4, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
rf_model = run_model_experiment(rf_model,
                                x_train,
                                y_train,
                                x_test,
                                y_test,
                                config = params,
                                name = 'RFwithOHE',
                                tags = ['RF', 'OheEncoding'])

wandb: 
wandb: Plotting RFwithOHE.
wandb: Logged summary metrics.
wandb: Logged learning curve.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


r2,▁
rmse,▁
r2,0.83415
rmse,0.85642


### RandomForest with Target Encoding

In [ ]:
target_encoder = TargetEncoder(cols = cat_features)

x_train, x_test, y_train, y_test = prepare_data(cars_df[x_columns],
                                                cars_df.Price,
                                                cat_encoder = target_encoder,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 100,
           "max_depth": 10,
           "max_features": .2,
           "max_samples": 0.4 }

rf_model = RandomForestRegressor(**params)  

rf_model = run_model_experiment(rf_model,
                                x_train,
                                y_train,
                                x_test,
                                y_test,
                                config = params,
                                name = 'RFwithTargetEncoding',
                                tags = ['RF', 'TargetEncoding'])

wandb: 
wandb: Plotting RFwithTargetEncoding.
wandb: Logged summary metrics.
wandb: Logged learning curve.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


r2,▁
rmse,▁
r2,0.86741
rmse,0.76576


### Gradient Boosted Trees

In [ ]:
target_encoder = TargetEncoder(cols = cat_features)

x_train, x_test, y_train, y_test = prepare_data(cars_df[x_columns],
                                                cars_df.Price,
                                                cat_encoder = target_encoder,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 100,
           "max_depth": 4 }

gbm_model = GradientBoostingRegressor(**params)  

gbm_model = run_model_experiment(gbm_model,
                                 x_train,
                                 y_train,
                                 x_test,
                                 y_test,
                                 config = params,
                                 name = 'GBMwithTargetEncoding',
                                 tags = ['GBM', 'TargetEncoding'])

wandb: 
wandb: Plotting GBMwithTargetEncoding.
wandb: Logged summary metrics.
wandb: Logged learning curve.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


r2,▁
rmse,▁
r2,0.91134
rmse,0.62618


### Grid Search and Best Model Tracking

In [ ]:
target_encoder = TargetEncoder(cols = cat_features)

x_train, x_test, y_train, y_test = prepare_data(cars_df[x_columns],
                                                cars_df.Price,
                                                cat_encoder = target_encoder,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": [100, 200, 300],
           "max_depth": [4, 6] }

gbm_cv = GridSearchCV(GradientBoostingRegressor(),
                      param_grid = params,
                      cv = 5,
                      scoring = 'neg_mean_squared_error')  

gbm_cv.fit(x_train, y_train)         

GridSearchCV(cv=10, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_...,
                           

In [ ]:
gbm_cv.best_params_

{'loss': 'huber', 'max_depth': 4, 'n_estimators': 300}

In [ ]:
gbm_model = GradientBoostingRegressor(**gbm_cv.best_params_)  

gbm_model = run_model_experiment(gbm_model,
                                 x_train,
                                 y_train,
                                 x_test,
                                 y_test,
                                 config = gbm_cv.best_params_,
                                 name = 'GBMwithTargetEncoding',
                                 tags = ['GBM', 'TargetEncoding'])

wandb: 
wandb: Plotting GBMwithTargetEncoding.
wandb: Logged summary metrics.
wandb: Logged learning curve.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


r2,▁
rmse,▁
r2,0.90722
rmse,0.64058


### XGB Model

In [ ]:
import xgboost as xgb

In [ ]:
target_encoder = TargetEncoder(cols = cat_features)

x_train, x_test, y_train, y_test = prepare_data(cars_df[x_columns],
                                                cars_df.Price,
                                                cat_encoder = target_encoder,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 200,
           "max_depth": 6,
           "objective": 'reg:squarederror' }

xgb_regressor = xgb.XGBRegressor(**params)

wandb.init(project='usedcar', 
           config=params, 
           tags = ['XGB', 'TargetEncoding'])

xgb_regressor.fit(x_train, 
                  y_train, 
                  callbacks=[wandb.xgboost.wandb_callback()])

wandb.run.name = "XGBWithTargetEncoding"

rmse = np.sqrt(mean_squared_error(y_test, 
                                  xgb_regressor.predict(x_test)))

r2 = xgb_regressor.score(x_test, y_test)

wandb.log( {"rmse" : rmse, 
            "r2": r2} )
  
wandb.save()
wandb.finish()

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


r2,▁
rmse,▁
r2,0.9142
rmse,0.616


## Get Experiment Details

In [ ]:
api = wandb.Api()

all_runs = api.runs("manaranjan/usedcar", order="+summary_metrics.rmse")

for run in all_runs:
  print(f"Model Name: {run.name} and RMSE {run.summary.get('rmse')}")
  print(run.config)

Model Name: XGBWithTargetEncoding and RMSE 0.6160012713560885
{'max_depth': 6, 'objective': 'reg:squarederror', 'n_estimators': 200}
Model Name: GBMwithTargetEncoding and RMSE 0.626183385535465
{'max_depth': 4, 'n_estimators': 100}
Model Name: GBMwithTargetEncoding and RMSE 0.6405800756270669
{'loss': 'huber', 'max_depth': 4, 'n_estimators': 300}
Model Name: RFwithTargetEncoding and RMSE 0.7657624697958263
{'max_depth': 10, 'max_samples': 0.4, 'max_features': 0.2, 'n_estimators': 100}
Model Name: RFwithOHE and RMSE 0.8564247778732033
{'max_depth': 10, 'max_samples': 0.4, 'max_features': 0.2, 'n_estimators': 100}
Model Name: DecisionTreeWithOHE and RMSE 0.9457617713462364
{}
